# Lab SQL

## Requirements

execute the following command to install the required packages:

```bash
pip install -r requirements.txt
```

In [17]:
"""Import csv files to sqlite3 database"""
import sqlite3
import pandas

conn = sqlite3.connect('courses.db')
# read csv files
courses = pandas.read_csv('Courses.csv', sep=',' , encoding='ISO-8859-1')
teachers = pandas.read_csv('Teachers.csv', sep=',' , encoding='ISO-8859-1')
students = pandas.read_csv('Students.csv', sep=',' , encoding='ISO-8859-1')
students_courses = pandas.read_csv('Students_Courses.csv', sep=',' , encoding='ISO-8859-1')
# create tables
courses.to_sql('TBL_COURSES', conn, index=False, if_exists='replace')
teachers.to_sql('TBL_TEACHERS', conn, index=False, if_exists='replace')
students.to_sql('TBL_STUDENTS', conn, index=False, if_exists='replace')
students_courses.to_sql('TBL_STUDENTS_COURSES', conn, index=False, if_exists='replace')
conn.close()


In [76]:
"""Queries"""

print('Cuántos cursos diferentes hay:')
conn = sqlite3.connect('courses.db')
cursor = conn.cursor()
cursor.execute('SELECT COUNT(DISTINCT Curso) AS COURSES FROM TBL_COURSES')
result = cursor.fetchall()
for row in result:
    print(row)

print('Cuántos alumnos hay en cada curso:')    
query = '''
SELECT c.CURSO, COUNT(DISTINCT SC.EstudianteID) AS STUDENTS 
FROM TBL_STUDENTS_COURSES as SC
INNER JOIN TBL_COURSES as C ON SC."Curso ID" = C."Curso ID"
GROUP BY C.CURSO
'''
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row)    
print('Determinar a partir de la respuesta anterior cuál es el curso más popular:')
query = '''
SELECT c.CURSO, COUNT(DISTINCT SC.EstudianteID) AS STUDENTS 
FROM TBL_STUDENTS_COURSES as SC
INNER JOIN TBL_COURSES as C ON SC."Curso ID" = C."Curso ID"
GROUP BY C.CURSO
HAVING STUDENTS = (
    SELECT MAX(STUDENTS) AS MAX_STUDENTS
    FROM (
        SELECT c.CURSO, COUNT(DISTINCT SC.EstudianteID) AS STUDENTS 
        FROM TBL_STUDENTS_COURSES as SC
        INNER JOIN TBL_COURSES as C ON SC."Curso ID" = C."Curso ID"
        GROUP BY C.CURSO
        )
    )
'''
print('De ese curso más popular extraer la lista de alumnos:')
query = '''
SELECT C.CURSO, S.Name AS STUDENT
FROM TBL_STUDENTS as S
INNER JOIN TBL_STUDENTS_COURSES as SC ON S.EstudianteID = SC.EstudianteID
INNER JOIN TBL_COURSES AS C ON SC."Curso ID" = C."Curso ID"
INNER JOIN (
    SELECT SC."Curso ID" AS Curso_ID, COUNT(DISTINCT SC.EstudianteID) AS STUDENTS 
    FROM TBL_STUDENTS_COURSES as SC
    GROUP BY SC."Curso ID"
    HAVING STUDENTS = (
        SELECT MAX(STUDENTS)
        FROM (
            SELECT c.CURSO, COUNT(DISTINCT SC.EstudianteID) AS STUDENTS 
            FROM TBL_STUDENTS_COURSES as SC
            INNER JOIN TBL_COURSES as C ON SC."Curso ID" = C."Curso ID"
            GROUP BY C.CURSO
            )
        )
    ) as C ON SC."Curso ID" = C.Curso_ID
'''
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row)

print('Definir en función del año de ingreso cuántos años hace que esta cada estudiante en la universidad:')
query = '''
SELECT S.Name as STUDENT, S.FechaIngreso, strftime('%Y', 'now') - substr(S.FechaIngreso, length(S.FechaIngreso) - 3, 4) AS YEARS  
FROM TBL_STUDENTS AS S
'''
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row)

conn.close()



Cuántos cursos diferentes hay:
(30,)
Cuántos alumnos hay en cada curso:
('Animacion Audiovisual', 31)
('Arbitro de futbol', 8)
('Audio for Games', 50)
('Coaching Ontologico', 8)
('Compliance y lavado de dinero', 20)
('Comunicación de Directivos', 5)
('Curso de Liquidacion de Sueldos', 6)
('Data Science', 1)
('Debates de politica exterior', 42)
('Deep Learning', 3)
('Derecho penal ', 43)
('Desarrollo con Angular', 30)
('Director Tecnico de Futbol', 3)
('Economia para principiantes', 17)
('Empleabilidad de personas con discapacidad', 43)
('Gestion del Inversor Financiero', 1)
('Gestion del transporte', 8)
('Introducción al Idioma Japonés Técnico y Manga', 7)
('Liderazgo Estrategico', 5)
('Luminotecnia', 3)
('Marketing Digital', 4)
('Nutricion deportiva', 46)
('PNL', 29)
('Periodismo deportivo', 2)
('Power BI', 2)
('Programacion Web Inicial', 43)
('Psicologia aplicada a las ciencias del deporte', 1)
('SQL', 46)
('Seguridad Electrica', 6)
('UX', 28)
Determinar a partir de la respuesta ante